## Ucitavanje podataka

In [1]:
import pandas as pd 
import numpy as np
df = pd.read_csv(
        'elektroprecisceno.csv',
         index_col=0
     )
df

,godine,stariji,Pol,indikacija:,ukupno,trajanje,frekvenca,Blok,dimenzija,EDD,...,Sotalol,Digoksin,ASA,OAKT,Plavix,Statin,Implantiran,Vreme,"sinhroni,",Elektrokonverzija
Broj,,,,,,,,,,,,,,,,,,,,,
1,78,1,1,1,5,5,0,3,42,56,...,0,0,0,2,0,0,1,2,2,True
2,61,0,1,2,24,1,7,0,42,59,...,0,0,0,2,0,0,0,-1,2,True
3,65,1,1,1,4,4,0,3,47,55,...,0,0,0,2,0,0,1,3,2,True
4,64,0,1,1,288,2,7,0,50,50,...,0,0,0,2,0,0,0,-1,2,True
5,65,1,2,1,48,3,9,0,49,61,...,0,0,0,2,0,1,0,-1,2,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
646,61,0,1,2,2,2,0,3,36,54,...,0,0,0,2,0,0,1,2,2,True
647,58,0,1,2,12,2,0,3,45,57,...,0,0,0,3,0,1,1,2,2,True
648,72,1,2,1,20,20,6,0,43,66,...,0,1,0,3,0,0,0,-1,2,True


## Podela podataka na trening i test

In [2]:
from sklearn.model_selection import train_test_split,GridSearchCV

X = df.iloc[:,:-1]
y = df.iloc[:,-1] 

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=0, stratify=y, test_size=0.35)#

## Izbor parametara modela

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

pipe = Pipeline([
        ('scale', StandardScaler()),
        ('clf', SVC(random_state=0,class_weight='balanced'))
       ])

param_grid = [
  {'clf__kernel': ['linear','poly','rbf','sigmoid'],
   'clf__max_iter': [5000,-1],
   'clf__C':[0.1, 0.5, 1.0],
   'clf__degree':[1,3,6,9],
   'clf__gamma':['scale','auto'],
   'clf__coef0':[0,0.5,1,1.5],
  },
]

grid = GridSearchCV(pipe, param_grid=param_grid, cv=5, refit='balanced_accuracy',n_jobs=5, scoring= ['balanced_accuracy','recall_macro','roc_auc'])
grid.fit(X, y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scale', StandardScaler()),
                                       ('clf',
                                        SVC(cache_size=500,
                                            class_weight='balanced',
                                            probability=True,
                                            random_state=0))]),
             n_jobs=5,
             param_grid=[{'clf__C': [0.1, 0.5, 1.0],
                          'clf__coef0': [0, 0.5, 1, 1.5],
                          'clf__degree': [1, 3, 6, 9],
                          'clf__gamma': ['scale', 'auto'],
                          'clf__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
                          'clf__max_iter': [5000, -1]}],
             refit='balanced_accuracy',
             scoring=['balanced_accuracy', 'recall_macro', 'roc_auc'])

In [4]:
print(grid.best_params_)
print(grid.best_score_)

{'clf__C': 0.1, 'clf__coef0': 0.5, 'clf__degree': 1, 'clf__gamma': 'scale', 'clf__kernel': 'sigmoid', 'clf__max_iter': 5000}
0.5858974358974359


## Normalizacija podataka

In [5]:
scaler = StandardScaler()  
scaler = scaler.fit(X_train)
X_train = scaler.transform(X_train) 
X_test = scaler.transform(X_test)

## Primena parametara za kreiranje modela

In [6]:
#{'clf__C': 0.1, 'clf__coef0': 0.5, 'clf__degree': 1, 'clf__gamma': 'scale', 'clf__kernel': 'sigmoid', 'clf__max_iter': 5000}
# balanced accuracy: 0.5858974358974359

clf = SVC(random_state=0,
                  class_weight='balanced',
                  C=0.1,
                  coef0=0.5,
                  kernel='sigmoid',
                  max_iter=5000
         )
clf = clf.fit(X_train, y_train)

## Primena i kvalitet modela

In [8]:
from sklearn.metrics import confusion_matrix,classification_report, balanced_accuracy_score

y_pred = clf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test, y_pred))
print("Balanced accuracy score:", balanced_accuracy_score(y_test,y_pred))

[[ 3  3]
 [15 31]]
              precision    recall  f1-score   support

       False       0.17      0.50      0.25         6
        True       0.91      0.67      0.78        46

    accuracy                           0.65        52
   macro avg       0.54      0.59      0.51        52
weighted avg       0.83      0.65      0.71        52

Balanced accuracy score: 0.5869565217391304
